# Libraries

In [266]:
from pyspark.sql import SparkSession, Window, WindowSpec
from pyspark.sql.types import StringType, IntegerType, DateType, BooleanType, StructType, StructField, TimestampType, DoubleType
from pyspark.sql.functions import col, min, max, lead, lag, rank, dense_rank, expr, minute, unix_timestamp, month, day, when, sum, date_trunc, date_diff
from faker import Faker
from datetime import datetime
import random

# My Spark Session

In [267]:
spark = SparkSession.builder.getOrCreate()
fake = Faker()

# Generate Test DataFrame

In [268]:
num_records = 50
data = [
    (
        i + 1,                      # id
        fake.first_name(),          # first_name
        fake.last_name(),           # last_name
        random.randint(18, 80),     # age
        fake.email(),               # email
        fake.city()                 # city
    )
    for i in range(num_records)
]
columns = ["id", "first_name", "last_name", "age", "email", "city"]

test_data_frame = spark.createDataFrame(data, schema = columns)

In [269]:
test_data_frame.show()

+---+----------+----------+---+--------------------+------------------+
| id|first_name| last_name|age|               email|              city|
+---+----------+----------+---+--------------------+------------------+
|  1|   Deborah|Cunningham| 36|  wwhite@example.com|North Brianborough|
|  2|     Linda|  Anderson| 24|angela28@example.net|     Chambersburgh|
|  3|  Samantha|    Dorsey| 54|jonestracy@exampl...|       Williamfort|
|  4|   Adriana|     Brown| 69|  paul11@example.net|   Lake Janicestad|
|  5|      Paul|     Chang| 74|dawnstevens@examp...|        South Jose|
|  6|       Ann|    Walker| 19|robinsonjulie@exa...|          Reedfurt|
|  7|  Courtney|   Freeman| 67|  gchung@example.org|         Jamestown|
|  8|     Danny|   Alvarez| 60|baileyjoshua@exam...|     Martinborough|
|  9|     Scott|  Williams| 41|zjohnson@example.org|          New Mark|
| 10|      Mark|   Johnson| 75|  hbrown@example.com|      South Denise|
| 11|    Bailey|   Johnson| 34|donaldriley@examp...|       Ashle

# Repeated Payments [Stripe SQL Interview Question]
Stripe asked this tricky SQL interview question, about identifying any payments made at the same merchant with the same credit card for the same amount within 10 minutes of each other and reporting the count of such repeated payments.

- same merchant
- same credit card
- same amount

In [270]:
schema = StructType([
    StructField("transaction_id", IntegerType(), True),
    StructField("merchant_id", IntegerType(), True),
    StructField("credit_card_id", IntegerType(), True),
    StructField("amount", IntegerType(), True),
    StructField("transaction_timestamp", TimestampType(), True)
])

data = [
    (1, 101, 1, 100, datetime.strptime('09/25/2022 12:00:00', '%m/%d/%Y %H:%M:%S')),
    (2, 101, 1, 100, datetime.strptime('09/25/2022 12:08:00', '%m/%d/%Y %H:%M:%S')),
    (3, 101, 1, 100, datetime.strptime('09/25/2022 12:28:00', '%m/%d/%Y %H:%M:%S')),
    (4, 102, 2, 300, datetime.strptime('09/25/2022 12:00:00', '%m/%d/%Y %H:%M:%S')),
    (6, 102, 2, 400, datetime.strptime('09/25/2022 14:00:00', '%m/%d/%Y %H:%M:%S'))
]
question_1 = spark.createDataFrame(data, schema)

In [271]:
window_spec = Window.partitionBy(["merchant_id", "credit_card_id", "amount"]).orderBy("transaction_timestamp")
answer_1 = question_1\
.withColumn('prev_time', lag("transaction_timestamp").over(window_spec))\
.withColumn('time_diff',  (unix_timestamp(col('transaction_timestamp')) - unix_timestamp(col('prev_time'))) / 60)\
.filter("time_diff<=10")
answer_1.count()

1

## SQL Query

```sql

WITH
  payments AS (
    SELECT
      *,
    EXTRACT(EPOCH FROM 

            transaction_timestamp - LAG(transaction_timestamp) 
            
            OVER (PARTITION BY merchant_id, credit_card_id, amount ORDER BY transaction_timestamp)
            
            )/60
            AS minute_difference
    FROM
      transactions
  )
SELECT
  COUNT(merchant_id) AS payment_count
FROM
  payments
WHERE
  minute_difference <= 10;
```

## Alternate solution

In [272]:
# Self-join the DataFrame with itself
df_alias1 = question_1.alias("t1")
df_alias2 = question_1.alias("t2")

# Join with conditions
joined = df_alias1.join(
    df_alias2,
    (col("t1.merchant_id") == col("t2.merchant_id")) &
    (col("t1.credit_card_id") == col("t2.credit_card_id")) &
    (col("t1.amount") == col("t2.amount")) &

    (col("t2.transaction_timestamp") > col("t1.transaction_timestamp")) &
    (col("t2.transaction_timestamp") <= expr("t1.transaction_timestamp + interval 10 minutes")),
    
    how = "inner"
)

# Count repeated payments
repeated_count = joined.count()

joined.show()

print("Repeated payments within 10 minutes:", repeated_count)

+--------------+-----------+--------------+------+---------------------+--------------+-----------+--------------+------+---------------------+
|transaction_id|merchant_id|credit_card_id|amount|transaction_timestamp|transaction_id|merchant_id|credit_card_id|amount|transaction_timestamp|
+--------------+-----------+--------------+------+---------------------+--------------+-----------+--------------+------+---------------------+
|             1|        101|             1|   100|  2022-09-25 12:00:00|             2|        101|             1|   100|  2022-09-25 12:08:00|
+--------------+-----------+--------------+------+---------------------+--------------+-----------+--------------+------+---------------------+

Repeated payments within 10 minutes: 1


## SQL Query

```sql
select
  count(*)
from
  transactions as t1
  inner join transactions as t2 on t1.merchant_id = t2.merchant_id
  AND t1.credit_card_id = t2.credit_card_id
  AND t1.amount = t2.amount
  AND t2.transaction_timestamp > t1.transaction_timestamp
  AND t2.transaction_timestamp <= t1.transaction_timestamp + INTERVAL '10 minute';
```

# Recursive CTE Example

```sql
WITH RECURSIVE EmployeeHierarchy AS (
    -- Anchor member: Select the top-level employee(s)
    SELECT EmployeeID, FirstName, LastName, ManagerID, 0 AS Level
    FROM Employees
    WHERE ManagerID IS NULL -- Assuming the top manager has NULL ManagerID

    UNION ALL

    -- Recursive member: Join with the CTE itself to find subordinates
    SELECT e.EmployeeID, e.FirstName, e.LastName, e.ManagerID, eh.Level + 1
    FROM Employees e
    INNER JOIN EmployeeHierarchy eh ON e.ManagerID = eh.EmployeeID
)
SELECT *
FROM EmployeeHierarchy
ORDER BY Level, FirstName;
```

# Median Google Search Frequency [Google SQL Interview Question]

Google’s Marketing Team needed to add a simple statistic to their upcoming Superbowl Ad: the median number of searches made per year. You were given a summary table that tells you the number of searches made last year, write a query to report the median searches made per user.

```sql
WITH RECURSIVE expanded AS (
    -- Initialize the recursion with the original data
    SELECT searches, num_users, 1 AS user_index
    FROM searches
    WHERE num_users > 0

    UNION ALL

    -- Recursively add rows based on num_users
    SELECT e.searches, e.num_users, e.user_index + 1
    FROM expanded e
    WHERE e.user_index + 1 <= e.num_users
),
ordered AS (
  SELECT 
  searches, 
  ROW_NUMBER() OVER (ORDER BY searches) AS rn,
  COUNT(*) OVER () AS total_users
  FROM expanded
)
SELECT
    round(AVG(searches)::NUMERIC, 2) AS median
FROM ordered
WHERE rn IN (total_users/2, total_users/2 + 2);
```

# Monthly Merchant Balance [Visa SQL Interview Question]
Say you have access to all the transactions for a given merchant account. Write a query to print the cumulative balance of the merchant account at the end of each day, with the total balance reset back to zero at the end of the month. Output the transaction date and cumulative balance.



```sql
WITH daily_balances AS (
  SELECT
    DATE_TRUNC('day', transaction_date) AS transaction_day,
    DATE_TRUNC('month', transaction_date) AS transaction_month,
    SUM(CASE WHEN type = 'deposit' THEN amount
      WHEN type = 'withdrawal' THEN -amount END) AS balance
  FROM visatransactions
  GROUP BY 
    DATE_TRUNC('day', transaction_date),
    DATE_TRUNC('month', transaction_date)
)

SELECT
  transaction_day,
  SUM(balance) OVER (
    PARTITION BY transaction_month
    ORDER BY transaction_day) AS balance
FROM daily_balances
ORDER BY transaction_day;
```

Hint

    Row	Cumulative Sum
    1	106.66
    2	106.66 + 98.50 = 205.16
    3	106.66 + 98.50 + 50.00 = 255.16

In [273]:
schema = StructType([
    StructField("transaction_id", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("transaction_date", TimestampType(), True)
])

data = [
    (19153, 'deposit', 65.90, datetime.strptime('07/10/2022 10:00:00', '%m/%d/%Y %H:%M:%S')),
    (53151, 'deposit', 178.55, datetime.strptime('07/08/2022 10:00:00', '%m/%d/%Y %H:%M:%S')),
    (29776, 'withdrawal', 25.90, datetime.strptime('07/08/2022 10:00:00', '%m/%d/%Y %H:%M:%S')),
    (16461, 'withdrawal', 45.99, datetime.strptime('07/08/2022 10:00:00', '%m/%d/%Y %H:%M:%S')),
    (77134, 'deposit', 32.60, datetime.strptime('07/10/2022 10:00:00', '%m/%d/%Y %H:%M:%S'))
]


question_3 = spark.createDataFrame(data, schema=schema)

In [274]:
question_3.show()

+--------------+----------+------+-------------------+
|transaction_id|      type|amount|   transaction_date|
+--------------+----------+------+-------------------+
|         19153|   deposit|  65.9|2022-07-10 10:00:00|
|         53151|   deposit|178.55|2022-07-08 10:00:00|
|         29776|withdrawal|  25.9|2022-07-08 10:00:00|
|         16461|withdrawal| 45.99|2022-07-08 10:00:00|
|         77134|   deposit|  32.6|2022-07-10 10:00:00|
+--------------+----------+------+-------------------+



In [275]:
question_3_group1 = question_3\
    .withColumn('month', date_trunc("month", col("transaction_date")))\
    .withColumn('day', date_trunc("day", col("transaction_date")))

question_3_group1 = question_3_group1.groupBy(["month", "day"]).agg(
    sum(
        when(
            col("type") == "deposit", 
            col("amount")
        ).otherwise(
            -1*col("amount")
        )
        ).alias("balance")
)
question_3_window = Window.partitionBy("month").orderBy("day")
question_3_group1.show()
answer_3 = question_3_group1.withColumn(
    "balance", sum("balance").over(question_3_window)) \
    .select("day", "balance") \
    .orderBy("day")

answer_3.show()

+-------------------+-------------------+-------+
|              month|                day|balance|
+-------------------+-------------------+-------+
|2022-07-01 00:00:00|2022-07-10 00:00:00|   98.5|
|2022-07-01 00:00:00|2022-07-08 00:00:00| 106.66|
+-------------------+-------------------+-------+

+-------------------+-------+
|                day|balance|
+-------------------+-------+
|2022-07-08 00:00:00| 106.66|
|2022-07-10 00:00:00| 205.16|
+-------------------+-------+



# Server Utilization Time [Amazon SQL Interview Question]

Fleets of servers power Amazon Web Services (AWS). Senior management has requested data-driven solutions to optimize server usage.

Write a query that calculates the total time that the fleet of servers was running. The output should be in units of full days.

Assumptions:

- Each server might start and stop several times.
- The total time in which the server fleet is running can be calculated as the sum of each server's uptime.


``` sql
with running_time as  (
  select 
  server_id,
  session_status,
  status_time AS start_time,
  LEAD(status_time) OVER(partition by server_id order by status_time) as stop_time
  from server_utilization
)
SELECT 
DATE_PART('days', JUSTIFY_HOURS(SUM(stop_time - start_time))) AS total_uptime_days
FROM running_time
WHERE session_status = 'start' AND stop_time IS NOT NULL;
```

In [276]:
schema = StructType([
    StructField("server_id", IntegerType(), True),
    StructField("status_time", TimestampType(), True),
    StructField("session_status", StringType(), True)
])

# Create the data
data = [
    (1, datetime.strptime("2022-08-02 10:00:00", "%Y-%m-%d %H:%M:%S"), "start"),
    (1, datetime.strptime("2022-08-04 10:00:00", "%Y-%m-%d %H:%M:%S"), "stop"),
    (2, datetime.strptime("2022-08-17 10:00:00", "%Y-%m-%d %H:%M:%S"), "start"),
    (2, datetime.strptime("2022-08-24 10:00:00", "%Y-%m-%d %H:%M:%S"), "stop")
]

question_4 = spark.createDataFrame(data, schema)

question_4.show()

+---------+-------------------+--------------+
|server_id|        status_time|session_status|
+---------+-------------------+--------------+
|        1|2022-08-02 10:00:00|         start|
|        1|2022-08-04 10:00:00|          stop|
|        2|2022-08-17 10:00:00|         start|
|        2|2022-08-24 10:00:00|          stop|
+---------+-------------------+--------------+



In [277]:
question_4_window = Window.partitionBy("server_id").orderBy("status_time")

question_4 = question_4\
.withColumn('stop_time', lead("status_time").over(question_4_window))\
.withColumn('diff', date_diff(col("stop_time"),col("status_time")))\
.filter("session_status = 'start' AND stop_time is not null")


answer_4 = question_4.agg(sum(col('diff')).alias('total_diff'))


answer_4.show()

+----------+
|total_diff|
+----------+
|         9|
+----------+

